### calculate the monthly precipitation accumulation from the daily MSWEP data for the Pacific region 

In [1]:
%matplotlib inline

### os 
import os 
import sys

### datetimes 
from datetime import datetime, timedelta

### scipy 
import numpy as np 
import pandas as pd
import xarray as xr

### plotting 
from matplotlib import pyplot as plt

In [2]:
def roll_longitudes(dset, lon_name='lon'): 
    """
    roll the longitudes of a dataset so that it goes from 0 to 360
    instead of -180 to 180
    Parameters
    ----------
    dset : xarray.Dataset
        The input Dataset with the longitudes going from -180 to 180
    lon_name : str, optional
        The name of the longitude dimension, by default 'lon'
        
    Returns
    -------
    
    dset : xarray.Dataset 
        Dataset with rolled longitudes 
    """
    
    dset = dset.assign_coords({lon_name:(dset[lon_name] % 360)}).roll({lon_name:(dset.dims[lon_name] // 2)}, roll_coords=True)
    
    return dset

In [3]:
def preprocess(dset): 
    
    dset = dset.sortby('lat') 
    dset = roll_longitudes(dset) 
    dset = dset.sel(lat=slice(-40., 40.), lon=slice(110, 280))
    return dset

In [4]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd() 

In [ ]:
year_start = 1979 
year_end = 2022

In [5]:
dpath_NRT = pathlib.Path('/media/nicolasf/END19101/ICU/data/glo2ho/MSWEP280/NRT/Daily') 

In [6]:
dpath_past = pathlib.Path('/media/nicolasf/END19101/ICU/data/glo2ho/MSWEP280/Past/Daily/') 

In [7]:
opath = pathlib.Path('../data/MSWEP/monthly')

In [8]:
clim = xr.open_dataset('../data/MSWEP/monthly/climatology/Monthly_climatology_1991_2020.nc')

In [9]:
for year in np.arange(year_start, year_end + 1): 
    
    lfiles_past = list(dpath_past.glob(f"{year}???.nc"))
    
    lfiles_NRT = list(dpath_NRT.glob(f"{year}???.nc"))
    
    lfiles = lfiles_past + lfiles_NRT
    
    print(f"{year}: {len(lfiles)}")
    
    lfiles.sort()
    
    dset = xr.open_mfdataset(lfiles, preprocess=preprocess, engine='netcdf4')

    dset = dset.chunk({'time':-1, 'lat':'auto', 'lon':'auto'})
    
    dsetm = dset.resample(time='1M').mean()
    
    clim = xr.open_dataset('../data/MSWEP/monthly/climatology/Monthly_climatology_1991_2020.nc')  
    
    anoms = dsetm.groupby(dsetm.time.dt.month) - clim
    
    dsetm.to_netcdf(opath.joinpath(f'MSWEP_monthly_from_daily_{year}.nc'))
    
    anoms.to_netcdf(opath.joinpath(f'MSWEP_monthly_anoms_from_daily_{year}.nc'))
    
    dset.close()
    
    dsetm.close() 
    
    anoms.close() 

1979: 364
1980: 366
1981: 365
1982: 365
1983: 365
1984: 366
1985: 365
1986: 365
1987: 365
1988: 366
1989: 365
1990: 365
1991: 365
1992: 366
1993: 365
1994: 365
1995: 365
1996: 366
1997: 365
1998: 365
1999: 365
2000: 366
2001: 365
2002: 365
2003: 365
2004: 366
2005: 365
2006: 365
2007: 365
2008: 366
2009: 365
2010: 365
2011: 365
2012: 366
2013: 365
2014: 365
2015: 365
2016: 366
2017: 365
2018: 365
2019: 365
2020: 400
2021: 365
2022: 365
